In [5]:
import torch
import pickle
import numpy as np
from models.alex import LeNetDropOut as model_alex
from models.artem import Net5 as model_artem
from models.gleb import LeNet as model_gleb
from models.kirill import Model5 as model_kirill
from models.nastya import Lenet4 as model_nastya

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

file = open('dataset.pkl', 'rb')
test_dataset = pickle.load(file)
file.close()

test_data = test_dataset['data'].numpy()
test_data = np.flip(test_data, axis = 3)
test_data = np.rot90(test_data, k=1, axes=(2,3))
test_data = 1 - test_data
test_data = torch.from_numpy(test_data).float().to(device)
test_labels = test_dataset['targets'].to(device)

In [8]:
device

device(type='cpu')

In [13]:
net1 = model_artem(0.1).to(device)
net1.load_state_dict(torch.load(combined_data, map_location=device))
net1.eval()
test_preds = torch.sigmoid(net1.forward(test_data))
print(((test_preds.argmax(dim=1) == test_labels).float().mean().cpu().data))

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 823739392 bytes.

In [1]:
# Определите путь к вашему исходному файлу
input_file_path = 'weights\Artem_ModelParameters0.8530000448226929.pth'

# Определите пути для двух выходных файлов
output_file_path_part1 = 'your_model_weights_part1.pth'
output_file_path_part2 = 'your_model_weights_part2.pth'

# Определите размер каждой части (в байтах)
chunk_size = 100000000  # например, 100 МБ

with open(input_file_path, 'rb') as input_file:
    # Прочитайте данные из исходного файла
    data = input_file.read()

    # Разделите данные на две части
    data_part1 = data[:chunk_size]
    data_part2 = data[chunk_size:]

    # Запишите данные в выходные файлы
    with open(output_file_path_part1, 'wb') as output_file_part1:
        output_file_part1.write(data_part1)

    with open(output_file_path_part2, 'wb') as output_file_part2:
        output_file_part2.write(data_part2)


In [2]:
def combine_files(input_file_path_part1, input_file_path_part2):
    with open(input_file_path_part1, 'rb') as input_file_part1, \
         open(input_file_path_part2, 'rb') as input_file_part2:

        # Читаем данные из первой и второй частей
        data_part1 = input_file_part1.read()
        data_part2 = input_file_part2.read()

        # Объединяем данные
        combined_data = data_part1 + data_part2

    return combined_data

# Определите пути для ваших файлов
input_file_path_part1 = 'your_model_weights_part1.pth'
input_file_path_part2 = 'your_model_weights_part2.pth'

# Вызовите функцию для объединения данных в одну переменную
combined_data = combine_files(input_file_path_part1, input_file_path_part2)


In [9]:
from io import BytesIO

net1 = model_artem().to(device)
net1.load_state_dict(torch.load(BytesIO(combined_data), map_location=device))
net1.eval()
test_preds = torch.sigmoid(net1.forward(test_data[:5000]))
print(((test_preds.argmax(dim=1) == test_labels[:5000]).float().mean().cpu().data))

tensor(0.8954)


In [ ]:
0.853